# Path e librerie
## Prima di modificare avvertire gli altri!

## Install

In [1]:
from google.colab import drive
drive_dir = 'drive' ; drive.mount('/content/drive')

Mounted at /content/drive


## Import

In [2]:
import tweepy
import numpy as np
from tweepy import Stream, OAuthHandler
from tqdm.notebook import tqdm
# from tqdm import tqdm !! per Jupyter from tqdm.notebook non funziona, runnare questo
import pandas as pd
import io
import pprint
import os
import glob
import warnings
warnings.filterwarnings('ignore')

## Path

In [3]:
##########################
### ROOT
##########################

prj_root_dir = f'/content/{drive_dir}/MyDrive/nucleare/'
os.makedirs(prj_root_dir, exist_ok=True)

##########################
### SCRAPING
##########################

scrapingATP_root_dir = prj_root_dir + 'scrapingATP/'
os.makedirs(scrapingATP_root_dir, exist_ok=True)

##########################
### ETL
##########################

etl_root_dir = prj_root_dir + 'ETL/'
os.makedirs(etl_root_dir, exist_ok=True)

##########################
### DWH
##########################

dw_root_dir = prj_root_dir + 'DWH/'
os.makedirs(dw_root_dir, exist_ok=True)

##########################
### SENTIMENT
##########################

dw_root_dir = prj_root_dir + 'SENTIMENT/'
os.makedirs(dw_root_dir, exist_ok=True)

##########################
### WORDEMBEDDING
##########################

dw_root_dir = prj_root_dir + 'EMBEDDING/'
os.makedirs(dw_root_dir, exist_ok=True)

##########################
### OUTPUT
##########################

dw_root_dir = prj_root_dir + 'OUTPUT/'
os.makedirs(dw_root_dir, exist_ok=True)

### Path per Jupyter
Solo quando si usa Jupyter runnare, il percorso dovrebbe essere lo stesso per tutti/e

In [ ]:
##########################
### ROOT
##########################

drive_dir = f'G:\\My Drive'
prj_root_dir = f'{drive_dir}\\nucleare\\'

##########################
### SCRAPING
##########################

scrapingATP_root_dir = prj_root_dir + 'scrapingATP\\'

##########################
### ETL
##########################

etl_root_dir = prj_root_dir + 'ETL\\'

##########################
### DWH
##########################

dw_root_dir = prj_root_dir + 'DWH\\'

##########################
### SENTIMENT
##########################

dw_root_dir = prj_root_dir + 'SENTIMENT\\'

##########################
### WORDEMBEDDING
##########################

dw_root_dir = prj_root_dir + 'EMBEDDING\\'

##########################
### OUTPUT
##########################

dw_root_dir = prj_root_dir + 'OUTPUT\\'

# 1. Ingest
## Antonello

cartella di scraping: nulceare/scraping

## Codici Twitter

In [ ]:
consumer_key = 'MQXChIn29dCky3pOQyW890Uxi'
consumer_secret = 'u4n3UeFz58b4H6InWFhBDUPZsfJ8xdVS0wODdnfAjz6e6PUbCJ'
access_token = '262247836-oWkm6AZIw4K0Fldnt8VBXwuL9IbtNCZwVj73ODkI'
access_secret = 'GySGl7BGaYvmXUtCapu5pHwEDHD5tA3Qd4yCMIDPKm1Wx'
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

# consumer_key = 'Vzk6O5t967f1n4iz9iQcsQwhf'
# consumer_secret = 'Q8sncdY1j85mwNitOqb5Spcg6U6uuZ0E2KNHHZEYF2J4DdCyVi'
# access_token = '164224789-mFmU5YhessCNvubNojgfL605tbU0vh0NOna3HdQU'
# access_secret = 'Ic5LNOH2QqxziSq4eTQdJvbXo3noTcQDUzCKXgFcdPzpC'
# auth = OAuthHandler(consumer_key, consumer_secret)
# auth.set_access_token(access_token, access_secret)
# api = tweepy.API(auth, wait_on_rate_limit=True)

# consumer_key = 'A4GRiWdUDgiGPckQSykzA'
# consumer_secret = 'nSyjpFDTr3eEmd8tFeBM1lwVrTNPLaHrcx0DicTEEuw'
# access_token = '1678747572-lbkg1jxpe5HypoTRSnRMeiVIe0RBMxhdjLJ0GJ6'
# access_secret = '1qVP0pHXV9HDgKhHDIy2HHqqnHFBm3sI5AR2ZDwZxlM'
# auth = OAuthHandler(consumer_key, consumer_secret)
# auth.set_access_token(access_token, access_secret)
# api = tweepy.API(auth, wait_on_rate_limit=True)

## Scraping dello storico

In [ ]:
utente = 'antonello' #mettere il proprio nome
lang = 'all' #mettere la lingua
prog = '08' #mettere il progressivo

#mettere la query
query_00 = ('@janniksin' or '@AlexZverev' or '@DjokerNole' or '@DaniilMedwed' or '@HubertHurkacz' or '@steftsitsipas' or '@CasperRuud98' or '@AndreyRublev97' or 'NittoATPFinals' or 'TennisTakesOverTorino' or '#ATPFinals' or 'ATPFinals' or 'ATP Finals' or 'Djokovic' or 'Medvedev' or 'Zverev' or 'Tsitsipas' or 'Rublev' or 'Berrettini' or 'Hurkacz' or 'Ruud' or 'Sinner')
query_01 = ('Zverev' or 'Hurkacz' or '#NittoATPFinals' or 'NittoATPFinals' or 'TennisTakesOverTorino' or '#ATPFinals' or 'ATPFinals' or 'ATP Finals')

In [ ]:
searched_tweets = []
last_id = -1
max_tweets = 500_000
pbar = tqdm(total=max_tweets)

while len(searched_tweets) < max_tweets:
    count = max_tweets - len(searched_tweets)
    try:
        # api.search è un metodo che su colab funziona anche se non è documentato in Tweepy. Su Jupyter non funziona, sostituire con search_tweets
        new_tweets = api.search(q=query_01, count=1000, max_id=str(last_id - 1), tweet_mode="extended") # tweet_mode="extended" importante, altrimenti il testo del tweet viene troncato
        # If I've reached the end of the search then I'm done
        # immediately the cycle while
        if not new_tweets:
            break
        # added the data found to the list
        searched_tweets.extend(new_tweets)
        # tqdm
        pbar.update(len(new_tweets))
        # retrieve the last id found
        last_id = new_tweets[-1].id
    except tweepy.TweepError as e:
        print(e)
        break

pbar.close()

  0%|          | 0/500000 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
print('Totale Tweet')
len(searched_tweets)

Totale Tweet


12199

In [ ]:
dict_ = {'id': [], 'user': [], 'date': [], 'text': [], 
         'favorite_count': [], 'hashtags': [], 
         'location': [], 'retweet': [], 'retweet_count': [], 'followers_count': [], 'in_reply_to_status_id':[], 'user_mentions':[]}

for status_j in searched_tweets[0:max_tweets]:
    status = status_j._json

    if 'retweeted_status' in status:
      if 'extended_tweet' in status['retweeted_status']:
        text = status['retweeted_status']['extended_tweet']['full_text']
        dict_['text'].append(text)
      else:
        text = status['retweeted_status']['full_text']
        dict_['text'].append(text)
    else:
      if 'extended_tweet' in status:
        text = status['extended_tweet']['full_text']
        dict_['text'].append(text)
      else:
        text = status['full_text']
        dict_['text'].append(text)
    
    dict_['id'].append(status['id'])
    dict_['user'].append(status['user']['screen_name'])
    dict_['hashtags'].append([hashtag['text'] for hashtag in status['entities']['hashtags']])
    dict_['user_mentions'].append([user_mention['screen_name'] for user_mention in status['entities']['user_mentions']])
    dict_['location'].append(status['user']['location'])
    dict_['followers_count'].append(status['user']['followers_count'])
    dict_['in_reply_to_status_id'].append(status['in_reply_to_status_id'])
    dict_['date'].append(status['created_at'])
    dict_['favorite_count'].append(status['favorite_count'])
    dict_['retweet_count'].append(status['retweet_count'])
    dict_['retweet'].append(status['retweeted'])

df = pd.DataFrame.from_dict(dict_, orient="columns")
df = df.set_index("id")
df.sort_values(by=['date'], inplace=True, ascending=True)
df.head(10)

,user,date,text,favorite_count,hashtags,location,retweet,retweet_count,followers_count,in_reply_to_status_id,user_mentions
id,,,,,,,,,,,
1460034895997243394,ValdoOlga,Mon Nov 15 00:00:19 +0000 2021,La “firma” sulla telecamera di Zverev.\nE le p...,0,[],,False,46,120,NaN,[toMMilanello]
1460035000661860360,Zverev_fans,Mon Nov 15 00:00:44 +0000 2021,☹️☹️☹️\n\nAll of us after Berrettini is forced...,0,[NittoATPFinals],,False,173,417,NaN,[TennisTV]
1460035118458888198,RaiSport,Mon Nov 15 00:01:13 +0000 2021,#Berrettini costretto ad abbandonare il match ...,52,"[Berrettini, Zverev, LaDS]",,False,6,247865,NaN,[]
1460035123047723010,mantas____,Mon Nov 15 00:01:14 +0000 2021,Not how any of us wanted this one to end 😢\n\n...,0,[NittoATPFinals],,False,304,6,NaN,[TennisTV]
1460035259551162373,Cucciolina96251,Mon Nov 15 00:01:46 +0000 2021,L’infortunio di Matteo #Berrettini mi distrugg...,0,[Berrettini],,False,23,122,NaN,[Alenize82]
1460035274663211012,Zverev_fans,Mon Nov 15 00:01:50 +0000 2021,'I'm sure it's not the last time we're gonna s...,0,[],,False,41,417,NaN,"[TennisTV, AlexZverev]"
1460035452744970244,malaymail,Mon Nov 15 00:02:32 +0000 2021,"Medvedev fights back to sink Hurkacz, Berretti...",1,[],Malaysia,False,0,267422,NaN,[]
1460035535997652999,nerdswire,Mon Nov 15 00:02:52 +0000 2021,Zverev gewinnt Auftakt bei ATP Finals https://...,0,[],"NRW, Deutschland",False,0,1457,NaN,[]
1460035698422075392,Martii063,Mon Nov 15 00:03:31 +0000 2021,Alexander Zverev vainqueur par abandon de Matt...,0,[],France,False,0,598,NaN,[]


In [ ]:
df = pd.DataFrame.from_dict(dict_, orient="columns")
df = df.set_index("id")
df.sort_values(by=['date'], inplace=True, ascending=True)
df.head(10)

,user,date,text,favorite_count,hashtags,location,retweet,retweet_count,followers_count,in_reply_to_status_id,user_mentions
id,,,,,,,,,,,
1460034895997243394,ValdoOlga,Mon Nov 15 00:00:19 +0000 2021,La “firma” sulla telecamera di Zverev.\nE le p...,0,[],,False,46,120,NaN,[toMMilanello]
1460035000661860360,Zverev_fans,Mon Nov 15 00:00:44 +0000 2021,☹️☹️☹️\n\nAll of us after Berrettini is forced...,0,[NittoATPFinals],,False,173,417,NaN,[TennisTV]
1460035118458888198,RaiSport,Mon Nov 15 00:01:13 +0000 2021,#Berrettini costretto ad abbandonare il match ...,52,"[Berrettini, Zverev, LaDS]",,False,6,247865,NaN,[]
1460035123047723010,mantas____,Mon Nov 15 00:01:14 +0000 2021,Not how any of us wanted this one to end 😢\n\n...,0,[NittoATPFinals],,False,304,6,NaN,[TennisTV]
1460035259551162373,Cucciolina96251,Mon Nov 15 00:01:46 +0000 2021,L’infortunio di Matteo #Berrettini mi distrugg...,0,[Berrettini],,False,23,122,NaN,[Alenize82]
1460035274663211012,Zverev_fans,Mon Nov 15 00:01:50 +0000 2021,'I'm sure it's not the last time we're gonna s...,0,[],,False,41,417,NaN,"[TennisTV, AlexZverev]"
1460035452744970244,malaymail,Mon Nov 15 00:02:32 +0000 2021,"Medvedev fights back to sink Hurkacz, Berretti...",1,[],Malaysia,False,0,267422,NaN,[]
1460035535997652999,nerdswire,Mon Nov 15 00:02:52 +0000 2021,Zverev gewinnt Auftakt bei ATP Finals https://...,0,[],"NRW, Deutschland",False,0,1457,NaN,[]
1460035698422075392,Martii063,Mon Nov 15 00:03:31 +0000 2021,Alexander Zverev vainqueur par abandon de Matt...,0,[],France,False,0,598,NaN,[]


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12199 entries, 1460034895997243394 to 1461120978436734981
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   user                   12199 non-null  object 
 1   date                   12199 non-null  object 
 2   text                   12199 non-null  object 
 3   favorite_count         12199 non-null  int64  
 4   hashtags               12199 non-null  object 
 5   location               12199 non-null  object 
 6   retweet                12199 non-null  bool   
 7   retweet_count          12199 non-null  int64  
 8   followers_count        12199 non-null  int64  
 9   in_reply_to_status_id  1645 non-null   float64
 10  user_mentions          12199 non-null  object 
dtypes: bool(1), float64(1), int64(3), object(6)
memory usage: 1.0+ MB


In [ ]:
print(df.shape)

(12199, 11)


In [ ]:
df.to_csv(f'{scrapingATP_root_dir}twitter_history_{utente}_{prog}_{lang}_{len(searched_tweets)}.csv')

## Scraping dello stream

In [ ]:
class MyStreamListener(tweepy.StreamListener):
    data = [] #qui ci metto i tweet che scarico man mano
    num = 0 #questo è un contatore settato a 0

    def store(self):
      dict_ = {'id': [], 'user': [], 'date': [], 'text': [],
         'favorite_count': [], 'hashtags': [], 
         'location': [], 'retweet': [], 'retweet_count': [], 'followers_count': [], 'in_reply_to_status_id':[], 'user_mentions':[]}
      
      for status_j in self.data:
          status = status_j._json

          if 'retweeted_status' in status:
            if 'extended_tweet' in status['retweeted_status']:
              text = status['retweeted_status']['extended_tweet']['full_text']
              dict_['text'].append(text)
            else:
              text = status['retweeted_status']['full_text']
              dict_['text'].append(text)
          else:
            if 'extended_tweet' in status:
              text = status['extended_tweet']['full_text']
              dict_['text'].append(text)
            else:
              text = status['full_text']
              dict_['text'].append(text)

          dict_['id'].append(status['id'])
          dict_['user'].append(status['user']['screen_name'])
          dict_['hashtags'].append([hashtag['text'] for hashtag in status['entities']['hashtags']])
          dict_['user_mentions'].append([user_mention['screen_name'] for user_mention in status['entities']['user_mentions']])
          dict_['location'].append(status['user']['location'])
          dict_['followers_count'].append(status['user']['followers_count'])
          dict_['in_reply_to_status_id'].append(status['in_reply_to_status_id'])
          dict_['date'].append(status['created_at'])
          dict_['full_text'].append(status['full_text'])
          dict_['favorite_count'].append(status['favorite_count'])
          dict_['retweet_count'].append(status['retweet_count'])
          dict_['retweet'].append(status['retweeted'])

      df = pd.DataFrame.from_dict(dict_, orient="columns")
      df = df.set_index("id")
      df.sort_values(by='favorite_count', inplace=True, ascending=False)
      df.to_csv(f'{scrapingATP_root_dir}_twitter_stream_{utente}_{prog}_{lang}_' + str(self.num) + '.csv') #salvo in csv
      self.num = self.num + 1
      self.data = []


    def on_status(self, status):
        self.data.append(status)
        # to pandas
        # store su google drive
        if len(self.data) > 5000:
          self.store()

time: 37.9 ms (started: 2021-11-15 22:17:42 +00:00)


In [ ]:
myStreamListener = MyStreamListener()
myStream = tweepy.Stream(auth = api.auth, listener=myStreamListener, tweet_mode="extended")

time: 2.06 ms (started: 2021-11-15 22:17:45 +00:00)


In [ ]:
query = ['NittoATPFinals' or '#ATPFinals' or 'ATPFinals' or 'TennisTakesOverTorino' or 'Djokovic' or 'Medvedev' or 'Zverev' or 'Tsitsipas' or 'Rublev' or 'Berrettini' or 'Hurkacz' or 'Ruud' or 'Sinner']
myStream.filter(track=query)

KeyboardInterrupt: ignored

time: 4.37 s (started: 2021-11-15 22:17:51 +00:00)


In [ ]:
len(myStreamListener.data)

0

time: 4.37 ms (started: 2021-11-15 22:18:02 +00:00)


In [ ]:
dict_ = {'id': [], 'user': [], 'date': [], 'text': [], 
         'favorite_count': [], 'hashtags': [], 
         'location': [], 'retweet': [], 'retweet_count': [], 'followers_count': [], 'in_reply_to_status_id':[], 'user_mentions':[]}
#
for status_j in myStreamListener.data:
    status = status_j._json

    if 'retweeted_status' in status:
      if 'extended_tweet' in status['retweeted_status']:
        text = status['retweeted_status']['extended_tweet']['full_text']
        dict_['text'].append(text)
      else:
        text = status['retweeted_status']['text']
        dict_['text'].append(text)
    else:
      if 'extended_tweet' in status:
        text = status['extended_tweet']['full_text']
        dict_['text'].append(text)
      else:
        text = status['text']
        dict_['text'].append(text)

    dict_['id'].append(status['id'])
    dict_['user'].append(status['user']['screen_name'])
    dict_['hashtags'].append([hashtag['text'] for hashtag in status['entities']['hashtags']])
    dict_['user_mentions'].append([user_mention['screen_name'] for user_mention in status['entities']['user_mentions']])
    dict_['location'].append(status['user']['location'])
    dict_['followers_count'].append(status['user']['followers_count'])
    dict_['in_reply_to_status_id'].append(status['in_reply_to_status_id'])
    dict_['date'].append(status['created_at'])
    dict_['favorite_count'].append(status['favorite_count'])
    dict_['retweet_count'].append(status['retweet_count'])
    dict_['retweet'].append(status['retweeted'])

In [ ]:
df = pd.DataFrame.from_dict(dict_, orient="columns")
df = df.set_index("id")
df.sort_values(by='favorite_count', inplace=True, ascending=False)
df.head(10)

,user,date,text,favorite_count,hashtags,location,retweet,retweet_count,followers_count,in_reply_to_status_id,user_mentions
id,,,,,,,,,,,
1460003463719440388,TennisTV,Sun Nov 14 21:55:25 +0000 2021,Not how any of us wanted this one to end 😢\n\n...,2696,[NittoATPFinals],Watch ATP Tour tournaments 👉,False,304,420515,NaN,[]
1460635739532582912,atptour,Tue Nov 16 15:47:52 +0000 2021,Relentless 💪 \n\n🇷🇺 @DaniilMedwed battles past...,1832,[NittoATPFinals],Follow the ATP Tour ➡,False,188,1690836,NaN,[DaniilMedwed]
1460003037032824836,atptour,Sun Nov 14 21:53:44 +0000 2021,💔\n\nMatteo Berrettini is forced to retire aga...,1824,[NittoATPFinals],Follow the ATP Tour ➡,False,146,1690836,NaN,[]
1460005336006463488,ItaliaTeam_it,Sun Nov 14 22:02:52 +0000 2021,A causa di un infortunio Matteo Berrettini è c...,1695,"[NittoATPFinals, ItaliaTeam]",Italia,False,158,103925,NaN,"[MattBerrettini, federtennis]"
1460006164935200772,FBiasin,Sun Nov 14 22:06:09 +0000 2021,Gran bel gesto di #Zverev.\n\n#Berrettini http...,1554,"[Zverev, Berrettini]",,False,53,141643,NaN,[]
1460002788948287489,TennisTV,Sun Nov 14 21:52:45 +0000 2021,Devastating 😔\n\nMatteo Berrettini pulls up wi...,1315,[NittoATPFinals],Watch ATP Tour tournaments 👉,False,135,420515,NaN,[]
1460003575149604869,romeoagresti,Sun Nov 14 21:55:52 +0000 2021,Che bello il gesto di #Zverev —&gt; ☹️,869,[Zverev],Torino,False,38,134357,NaN,[]
1460621045644234757,atptour,Tue Nov 16 14:49:28 +0000 2021,Zverev's not done yet 😳 We're going 3!\n\nMedv...,697,[NittoATPFinals],Follow the ATP Tour ➡,False,57,1690836,NaN,[]
1461337322276855811,josemorgado,Thu Nov 18 14:15:42 +0000 2021,Medvedev-Sinner will be a dead rubber.\n\nAbso...,606,[],"Lisbon, Portugal",False,34,83821,NaN,[]


In [ ]:
df.to_csv(f'{scrapingATP_root_dir}twitter_stream_{utente}_{prog}_{lang}_{len(myStreamListener.data)}.csv') #scarico l'ultimo stream

In [ ]:
#unisco tutti i csv

path = scrapingATP_root_dir

all_files = glob.glob(os.path.join(path, "twitter_*.csv"))
df_from_each_file = (pd.read_csv(f, sep=',') for f in all_files)
df_merged = pd.concat(df_from_each_file, ignore_index=True)
df_merged.drop_duplicates(subset=['id'], inplace=True, ignore_index=False)
df_merged.to_csv(f'{etl_root_dir}twitter_merged_from_scraping.csv', index=False, sep=',', encoding='utf-8')

In [ ]:
df_merge = pd.read_csv(f'{etl_root_dir}twitter_merged_from_scraping.csv')
df_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33480 entries, 0 to 33479
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     33480 non-null  int64  
 1   user                   33480 non-null  object 
 2   date                   33480 non-null  object 
 3   text                   33480 non-null  object 
 4   favorite_count         33480 non-null  int64  
 5   hashtags               33480 non-null  object 
 6   location               19827 non-null  object 
 7   retweet                33480 non-null  bool   
 8   retweet_count          33480 non-null  int64  
 9   followers_count        33480 non-null  int64  
 10  in_reply_to_status_id  2999 non-null   float64
 11  user_mentions          33480 non-null  object 
dtypes: bool(1), float64(1), int64(4), object(6)
memory usage: 2.8+ MB


In [ ]:
df_merge.tail()

,id,user,date,text,favorite_count,hashtags,location,retweet,retweet_count,followers_count,in_reply_to_status_id,user_mentions
58124,1461119368969039882,Recupxrateur,Wed Nov 17 23:49:38 +0000 2021,🎯 | #ATPFinals \n\nZverev 2-0 | 1.67 | 1%\nMed...,0,"['ATPFinals', 'TeamParieur']",NaN,False,0,4,NaN,[]
58125,1461120133384994820,raihannb,Wed Nov 17 23:52:40 +0000 2021,Vigrin ATP: afraid to address Zverev's case\nC...,0,[],Australia,False,76,997,NaN,['_silver912']
58126,1461120695748997123,Marcelo21142,Wed Nov 17 23:54:54 +0000 2021,#NittoATPFinals Day 5 Media Notes: https://t.c...,0,['NittoATPFinals'],RJ,False,18,153,NaN,"['ATPMediaInfo', 'DaniilMedwed', 'janniksin']"
58127,1461120927844999172,MnardShady,Wed Nov 17 23:55:50 +0000 2021,"@SG_Lockwood Bah ecoute, chacun son delire. De...",0,[],"Paris, France",False,0,71,1.461121e+18,['SG_Lockwood']
58128,1461120978436734981,PokornyRoland,Wed Nov 17 23:56:02 +0000 2021,@TheBorisBecker \n\nHi Boris.\n\nwill you comm...,0,[],NaN,False,0,15,NaN,['TheBorisBecker']


# 2. ETL
**NATALIA**



1.   rimuovere #
2.   rimuovere id doppi
1.   rimuovere http
2.   rimuovere twitter con testo vuoto
1.   rimuovere @



file input: nucleare/ETL/twitter_merged_from_scraping.csv

cartella output: nucleare/DWH/

In [ ]:
file_path = '/content/drive/MyDrive/nucleare/'
df = pd.read_csv(file_path+'ETL/twitter_merged_from_scraping.csv')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33480 entries, 0 to 33479
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     33480 non-null  int64  
 1   user                   33480 non-null  object 
 2   date                   33480 non-null  object 
 3   text                   33480 non-null  object 
 4   favorite_count         33480 non-null  int64  
 5   hashtags               33480 non-null  object 
 6   location               19827 non-null  object 
 7   retweet                33480 non-null  bool   
 8   retweet_count          33480 non-null  int64  
 9   followers_count        33480 non-null  int64  
 10  in_reply_to_status_id  2999 non-null   float64
 11  user_mentions          33480 non-null  object 
dtypes: bool(1), float64(1), int64(4), object(6)
memory usage: 2.8+ MB


In [ ]:
df.isna().sum()

id                           0
user                         0
date                         0
text                         0
favorite_count               0
hashtags                     0
location                 13653
retweet                      0
retweet_count                0
followers_count              0
in_reply_to_status_id    30481
user_mentions                0
dtype: int64

In [ ]:
df_clean=df

In [ ]:
df_clean = df_clean.drop_duplicates(subset='id', keep="first")

In [ ]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33480 entries, 0 to 33479
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     33480 non-null  int64  
 1   user                   33480 non-null  object 
 2   date                   33480 non-null  object 
 3   text                   33480 non-null  object 
 4   favorite_count         33480 non-null  int64  
 5   hashtags               33480 non-null  object 
 6   location               19827 non-null  object 
 7   retweet                33480 non-null  bool   
 8   retweet_count          33480 non-null  int64  
 9   followers_count        33480 non-null  int64  
 10  in_reply_to_status_id  2999 non-null   float64
 11  user_mentions          33480 non-null  object 
dtypes: bool(1), float64(1), int64(4), object(6)
memory usage: 3.1+ MB


In [ ]:
df_clean['text_cleaned']=df_clean['text']

In [ ]:
# rimuovo gli hashtag
df_clean['text_cleaned']=df_clean['text_cleaned'].replace('#\w+ ?','', regex=True)
#stringwithouthash = re.sub(r'#\w+ ?', '', tweet['text'])

In [ ]:
# rimuovo i link
df_clean['text_cleaned']=df_clean['text_cleaned'].replace('http\S+','', regex=True)
#stringwithoutlink = re.sub(r'http\S+', '', tweet['text'])

In [ ]:
# rimuovo le mention
df_clean['text_cleaned']=df_clean['text_cleaned'].replace('@\w+ ?','', regex=True)
#stringwithouthash = re.sub(r'@\w+ ?', '', tweet['text'])

In [ ]:
#rimuovo i numeri
df_clean['text_cleaned'] = df_clean['text_cleaned'].str.replace('\d+', '')

In [ ]:
df_clean['text_cleaned'] = df_clean['text_cleaned'].str.replace('\n','', regex=False)

In [ ]:
# rimuovo i doppi spazi
df_clean['text_cleaned'] = df_clean['text_cleaned'].str.replace('\s+','', regex=False)

In [ ]:
#df_clean['text_cleaned']=df_clean['text_cleaned'].str.replace('[^\w\s\d]',' ', regex=True)

In [ ]:
df_clean['text_cleaned']

0        Rublev’s revenge 😤reverses the result from the...
1        Red-hot start 🔥powers past  champion Tsitsipas...
2                                        Alexander Zverev 
3                                   When the 🐺 met the 🦁🤝 
4                                                  💙🤍💙🤍🇬🇷 
                               ...                        
33475    Alexander Zverev vs Hubert Hurkacz Odds &amp; ...
33476              🎯 | Zverev - | . | %Medvedev - | . | % 
33477    Vigrin ATP: afraid to address Zverev's caseCha...
33478    Bah ecoute, chacun son delire. Depuis le debut...
33479    Hi Boris.will you comment tomorrow match Zvere...
Name: text_cleaned, Length: 33480, dtype: object

In [ ]:
df_clean.isna().sum()

id                           0
user                         0
date                         0
text                         0
favorite_count               0
hashtags                     0
location                 13653
retweet                      0
retweet_count                0
followers_count              0
in_reply_to_status_id    30481
user_mentions                0
text_cleaned                 0
dtype: int64

In [ ]:
df_clean.to_csv(file_path+'DWH/twitter_text_cleaned.csv')

# 3. Sentiment
MARCO

# 4. BoW



In [ ]:
import warnings
import nltk ### libreria di linguistica (Stanford)
from tqdm import tqdm
from nltk.stem import WordNetLemmatizer #### individua la radice linguistica delle parole: non tronca le parole come un stamming
nltk.download('wordnet') ## tassonomia lingua inglese
nltk.download('stopwords')
from pprint import pprint
from gensim.models.phrases import Phrases
from nltk.corpus import stopwords
connectors = stopwords.words('english')

from nltk.corpus import stopwords
stop = stopwords.words('english')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
file_path = '/content/drive/MyDrive/nucleare/ETL/twitter_merged_from_scraping.csv'

df = pd.read_csv(file_path)

lemmatizer = WordNetLemmatizer()
warnings.filterwarnings('ignore')

text = df.text
print(len(text))
text = text[~(text.isna())]
####pulizia: sostituisco tutti i caratteri speciali con uno spazio
text = text.str.lower().str.replace('[^\w\s\d]',' ', regex=True) 
text = text.str.replace('\n','', regex=False) 
text = text.str.replace('\s+', ' ', regex=True) ## rimuovi spazi multipli
print(len(text))

###TOKENIZATION
token = text.str.split()

###LEMMATIZATION: recupero radice parole
ltoken = token.apply(lambda x: [lemmatizer.lemmatize(sent) for sent in x])

### STOP WORDS REMOVING
stop.extend(['good', 'bad', 'dont', 'many', 'love', 'excellent', 'would', 'perfect', 'even', 'great','nice', 'amazing'])
train_sentences = []
for row in ltoken:
    train_sentences.append([item for item in row if item not in stop])

ltoken = train_sentences

print(f'token[0]: {token[0]}')
print(f'ltoken[0]: {ltoken[0]}')

### BIGRAM
bigram = Phrases(ltoken, min_count=5, threshold=0.2, common_terms=connectors) #gensim 3, ad esempio "and" 
bigrams = [bigram[item] for item in ltoken]

### NGRAMS
ngram = Phrases(bigrams, min_count=5, threshold=0.2, common_terms=connectors) # gensim 3
ngrams = [ngram[item] for item in bigrams]

print(f'bigrams[0]: {bigrams[0]}')
print(f'ngrams[0]: {ngrams[0]}')

linearized_ngram_texts_list = [' '.join(ngrams_item) for ngrams_item in ngrams]
print(linearized_ngram_texts_list[1])

linearized_bgram_texts_list = [' '.join(bgrams_item) for bgrams_item in bigrams]
print(linearized_bgram_texts_list[1])

33480
33480
token[0]: ['rublev', 's', 'revenge', 'andreyrublev97', 'reverses', 'the', 'result', 'from', 'the', '2020', 'nittoatpfinals', 'and', 'gets', 'the', 'win', 'over', 'tsitsipas', 'https', 't', 'co', 'vqoi9wxpch']
ltoken[0]: ['rublev', 'revenge', 'andreyrublev97', 'revers', 'result', '2020', 'nittoatpfinals', 'get', 'win', 'tsitsipas', 'http', 'co', 'vqoi9wxpch']
bigrams[0]: ['rublev_revenge', 'andreyrublev97_revers', 'result_2020', 'nittoatpfinals_get', 'win_tsitsipas', 'http_co', 'vqoi9wxpch']
ngrams[0]: ['rublev_revenge_andreyrublev97_revers', 'result_2020_nittoatpfinals_get', 'win_tsitsipas_http_co', 'vqoi9wxpch']


In [ ]:
saving_path = '/content/drive/MyDrive/nucleare/tone/'
df_n  = pd.DataFrame(linearized_ngram_texts_list, columns=['ngrams'])
df_bi = pd.DataFrame(linearized_bgram_texts_list, columns=['bigrams'])

df_bi.to_csv(saving_path + 'bigrams.csv', index=False)
df_n.to_csv(saving_path + 'ngrams.csv', index=False)

df_bi.head()

####################################################
####################################################
 #### TO BE CONTINUED
####################################################
####################################################

,bigrams
0,rublev_revenge andreyrublev97_revers result_20...
1,red_hot start_andreyrublev97 power_past 2019_c...
2,alexander_zverev nittoatpfinals_http co nn9inn...
3,met_djokernole ibra_official_nittoatpfinals ht...
4,tsitsipas_nittoatpfinals http_co 8gfuvyyqiu


# 5. Word embeddings
TONE'

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
33480
33480


0        rublev s revenge andreyrublev97 reverses the r...
1        red hot start andreyrublev97 powers past 2019 ...
2        alexander zverev nittoatpfinals https t co nn9...
3        when the met the djokernole ibra_official nitt...
4           tsitsipas nittoatpfinals https t co 8gfuvyyqiu
                               ...                        
33475    alexander zverev vs hubert hurkacz odds amp pr...
33476     atpfinals zverev 2 0 1 67 1 medvedev 2 0 1 95...
33477    vigrin atp afraid to address zverev s casechad...
33478     sg_lockwood bah ecoute chacun son delire depu...
33479     theborisbecker hi boris will you comment tomo...
Name: text, Length: 33480, dtype: object

# 6. Lime

# "SocialNetwork Analisys"